In [1]:
from youtube_transcript_api import YouTubeTranscriptApi
import pandas as pd
import numpy as np
import requests
from random_user_agent.user_agent import UserAgent
import re
import json
import pickle
from bs4 import BeautifulSoup # library to parse HTML documents
# !pip install random-user-agent

In [2]:
playlist_url = 'https://www.youtube.com/playlist?list=PLzufeTFnhupwqGc-IVUPwviLVP0gsx8_w'

In [3]:
res = requests.get(playlist_url)

In [4]:
res

<Response [200]>

In [5]:
# res.text

In [6]:
playlist_text = res.text

In [7]:
video_ids = []

for el in re.findall(r'"videoId":".*?,', playlist_text):
    video_id = el.replace('"videoId":"', '')[:-2]
    if video_id not in video_ids:
        video_ids.append(el.replace('"videoId":"', '')[:-2])

In [8]:
# The first video is not an episode, so removing it

try:
    video_ids.remove('MOUxxMYYtBQ')
except:
    pass

In [9]:
len(video_ids)

35

In [10]:
# re.findall(r'"title":\{"runs":\[\{"text":".*?"\}\]', playlist_text)[0].replace('"title":', '') + "}"

titles = []

for el in re.findall(r'"title":\{"runs":\[\{"text":".*?"\}\]', playlist_text):
    try:
        titles.append(json.loads(el.replace('"title":', '') + "}")['runs'][0]['text'])
    except:
        pass

In [11]:
titles

['Abish ने लिया Sharks का एक Exclusive Interview | Shark Tank India| Getting Candid With The Sharks',
 'Knowledge Innovation | Shark Tank India - Ep 31 | Full Episode | 31 January 2022',
 'Annie: A Solution For Specially Abled | Shark Tank India - Ep 13 | Full Episode | 5 January 2022',
 'Women Taking Charge | Shark Tank India - Ep 10 | Full Episode | 31 December 2021',
 'Era Of Entrepreneurship | Shark Tank India - Ep 24 | Full Episode | 20 January 2022',
 'Matter Of Safety | Shark Tank India - Ep 9 | Full Episode | 30 December 2021',
 'Innovative Ideas | Shark Tank India - Ep 7 | Full Episode | 28 December 2021',
 'Beyond Food Taste | Shark Tank India - Ep 8 | Full Episode | 29 December 2021',
 "It's Time To Change | Shark Tank India - Ep 15 | Full Episode | 7 January 2022",
 'The Final Destination | Shark Tank India - Ep 35 | Full Episode | 4 February 2022',
 'Shark Tank - Welcome To Shark Tank India - Ep - 1 - Full Episode - 20th December, 2021',
 'A Desi Twist | Shark Tank India -

In [12]:
# Removing the first title which is not an episode and last few elements
if len(titles) > 35:
    titles = titles[1:36-len(titles)]

In [13]:
titles

['Knowledge Innovation | Shark Tank India - Ep 31 | Full Episode | 31 January 2022',
 'Annie: A Solution For Specially Abled | Shark Tank India - Ep 13 | Full Episode | 5 January 2022',
 'Women Taking Charge | Shark Tank India - Ep 10 | Full Episode | 31 December 2021',
 'Era Of Entrepreneurship | Shark Tank India - Ep 24 | Full Episode | 20 January 2022',
 'Matter Of Safety | Shark Tank India - Ep 9 | Full Episode | 30 December 2021',
 'Innovative Ideas | Shark Tank India - Ep 7 | Full Episode | 28 December 2021',
 'Beyond Food Taste | Shark Tank India - Ep 8 | Full Episode | 29 December 2021',
 "It's Time To Change | Shark Tank India - Ep 15 | Full Episode | 7 January 2022",
 'The Final Destination | Shark Tank India - Ep 35 | Full Episode | 4 February 2022',
 'Shark Tank - Welcome To Shark Tank India - Ep - 1 - Full Episode - 20th December, 2021',
 'A Desi Twist | Shark Tank India - Ep 34 | Full Episode | 3 February 2022',
 'Life-Changing Ideas | Shark Tank India - Ep 33 | Full Epis

In [14]:
len(titles)

35

In [15]:
ep_number_regex = r'[Ee][Pp](?:- | - | )(\d+)'

In [16]:
ep_numbers = [re.search(ep_number_regex, el).group(1) for el in titles]

In [17]:
df = pd.DataFrame([ep_numbers, video_ids, titles]).T

df.columns = ['Episode Number', 'VideoId', 'Title']

In [18]:
df.tail(3)

,Episode Number,VideoId,Title
32,4,8tR8LAsjJHY,Entrepreneurship's Wave | Shark Tank India - E...
33,3,WXuoYzyCAfw,Innovation Is The Investment | Shark Tank Indi...
34,2,qX2P0DNFiiM,Shark Tank - Snacktivity - EP- 2 - Full Episod...


In [19]:
df[df['Title'].str.contains('Welcome')]
# ['Title']

,Episode Number,VideoId,Title
9,1,OG-MwHvM4Ac,Shark Tank - Welcome To Shark Tank India - Ep ...


In [20]:
def get_subs(row):
    try:
        subs = YouTubeTranscriptApi.get_transcript(row['VideoId'])
        row['Subs'] = subs
    except Exception as e:
        print(f"Error :: {e} with video {row['VideoId']} and title {row['Title']}")

In [21]:
df['Subs'] = ""
for idx, row in df.iterrows():
    get_subs(row)

Error :: 
Could not retrieve a transcript for the video https://www.youtube.com/watch?v=DMOzm6rav68! This is most likely caused by:

Subtitles are disabled for this video

If you are sure that the described cause is not responsible for this error and that a transcript should be retrievable, please create an issue at https://github.com/jdepoix/youtube-transcript-api/issues. Please add which version of youtube_transcript_api you are using and provide the information needed to replicate the error. Also make sure that there are no open issues which already describe your problem! with video DMOzm6rav68 and title Knowledge Innovation | Shark Tank India - Ep 31 | Full Episode | 31 January 2022
Error :: 
Could not retrieve a transcript for the video https://www.youtube.com/watch?v=F5MKAtwVWZ4! This is most likely caused by:

Subtitles are disabled for this video

If you are sure that the described cause is not responsible for this error and that a transcript should be retrievable, please creat

In [22]:
mask = df['Subs'] == ""

df.loc[mask, 'Subs'] = np.empty((sum(mask), 0)).tolist()

In [23]:
df.tail(4)

,Episode Number,VideoId,Title,Subs
31,5,7Xyr1vT7xnc,Turning Ideas Into Reality | Shark Tank India ...,"[{'text': 'My name is Jayesh Tope.', 'start': ..."
32,4,8tR8LAsjJHY,Entrepreneurship's Wave | Shark Tank India - E...,[]
33,3,WXuoYzyCAfw,Innovation Is The Investment | Shark Tank Indi...,"[{'text': ''In 40 countries of this world'', '..."
34,2,qX2P0DNFiiM,Shark Tank - Snacktivity - EP- 2 - Full Episod...,"[{'text': ''Humans.'', 'start': 11.148, 'durat..."


In [24]:
df.to_pickle('Extracted_df.pickle')

## Cleaning YT Subtitles

In [25]:
def clean_subs(text_list):
    return ' '.join([el['text'].replace('\n', ' ').replace("'", "") for el in text_list])

In [26]:
df['Subs_Clean'] = df['Subs'].apply(clean_subs)

In [27]:
df.tail(4)

,Episode Number,VideoId,Title,Subs,Subs_Clean
31,5,7Xyr1vT7xnc,Turning Ideas Into Reality | Shark Tank India ...,"[{'text': 'My name is Jayesh Tope.', 'start': ...",My name is Jayesh Tope. My name is Pritesh Mah...
32,4,8tR8LAsjJHY,Entrepreneurship's Wave | Shark Tank India - E...,[],
33,3,WXuoYzyCAfw,Innovation Is The Investment | Shark Tank Indi...,"[{'text': ''In 40 countries of this world'', '...",In 40 countries of this world the show which g...
34,2,qX2P0DNFiiM,Shark Tank - Snacktivity - EP- 2 - Full Episod...,"[{'text': ''Humans.'', 'start': 11.148, 'durat...",Humans. Ideas. And dreams. This is a journey o...


In [28]:
# with open("Extracted_df.pickle", "rb") as f:
#     df2 = pickle.load(f)

## Extracting data from wikipedia

In [29]:
res = requests.get('https://en.wikipedia.org/wiki/Shark_Tank_India_(season_1)')

In [30]:
# res.text

In [31]:
soup = BeautifulSoup(res.text, 'html.parser')

table=soup.find('table',{'class':"wikitable mw-collapsible sortable"})

In [32]:
confirmed_symbol = '<td align="center" bgcolor="#9DE093"><img alt="Green checkmark" data-file-height="185" data-file-width="180" decoding="async" height="21" src="//upload.wikimedia.org/wikipedia/commons/thumb/4/41/Symbol_confirmed.svg/20px-Symbol_confirmed.svg.png" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/4/41/Symbol_confirmed.svg/30px-Symbol_confirmed.svg.png 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/4/41/Symbol_confirmed.svg/40px-Symbol_confirmed.svg.png 2x" width="20"/>'
# Replacing the check mark element shown on wikipedia representing if a judge has invested in the pitch 
# with 1.

table_str = str(table).replace(confirmed_symbol, "<td> 1")

In [33]:
df_wiki = pd.DataFrame(pd.read_html(table_str)[0])

df_wiki.columns = ['Ep. No.', 'Episode Number', 'Pitch No.', 'Brand', 'Idea', 'Original ask', \
                   'Deal', 'Ashneer', 'Namita', 'Anupam', 'Vineeta', 'Aman', 'Peyush', 'Ghazal']

In [34]:
try:
    df_wiki.drop(['Ep. No.'], axis=1, inplace=True)
except:
    pass

In [35]:
df_wiki.head(4)

,Episode Number,Pitch No.,Brand,Idea,Original ask,Deal,Ashneer,Namita,Anupam,Vineeta,Aman,Peyush,Ghazal
0,1,1,BluePine Industries,Frozen Momos,₹50 lakhs for 5% equity,₹75 lakhs for 16% equity,1,NaN,NaN,1,1,—,—
1,1,2,Booz scooters,Renting e-bike for mobility in private spaces,₹40 lakhs for 15% equity,₹40 lakhs for 50% equity,1,NaN,NaN,1,NaN,—,—
2,1,3,Heart up my Sleeves,Detachable Sleeves,₹25 lakhs for 10% equity,₹25 lakhs for 30% equity,NaN,NaN,1,1,NaN,—,—
3,2,4,Tagz Foods,Healthy Potato Chips,₹70 lakhs for 1% equity,₹70 lakhs for 2.75% equity,1,NaN,NaN,NaN,NaN,—,—


In [36]:
# Renaming the column names of df obtained from extracting subtitles from SET India Shark Tank Episodes

if not any(['YT' in column for column in df.columns]):
    df.columns = [f"YT_{column}" if column != 'Episode Number' else 'Episode Number' for column in df.columns]

In [37]:
final_df = df_wiki.merge(df, on='Episode Number', how='left')

In [38]:
final_df.tail(4)

,Episode Number,Pitch No.,Brand,Idea,Original ask,Deal,Ashneer,Namita,Anupam,Vineeta,Aman,Peyush,Ghazal,YT_VideoId,YT_Title,YT_Subs,YT_Subs_Clean
116,35,117,Elcare India,Carenting for Elders,₹1 crore for 2.5% Equity,No Deal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FavzJTUfddQ,The Final Destination | Shark Tank India - Ep ...,"[{'text': ''With all the seven sharks, today w...","With all the seven sharks, today we have Shark..."
117,35,118,Lenskart,Eyeware,IN POPULAR CULTURE,IN POPULAR CULTURE,IN POPULAR CULTURE,IN POPULAR CULTURE,IN POPULAR CULTURE,IN POPULAR CULTURE,IN POPULAR CULTURE,IN POPULAR CULTURE,IN POPULAR CULTURE,FavzJTUfddQ,The Final Destination | Shark Tank India - Ep ...,"[{'text': ''With all the seven sharks, today w...","With all the seven sharks, today we have Shark..."
118,Investment,Investment,Investment,Investment,Investment,Investment,₹5.383Cr,₹6.383Cr,₹5.338Cr,₹3.042Cr,₹9.358Cr,₹8.297Cr,₹1.2Cr,NaN,NaN,NaN,NaN
119,Number of Deals Made,Number of Deals Made,Number of Deals Made,Number of Deals Made,Number of Deals Made,Number of Deals Made,21,22,24,15,28,27,7,NaN,NaN,NaN,NaN


In [39]:
final_df.to_pickle('Shark Tank India S1 Data with Subtitles.pickle')

In [40]:
final_df.drop('YT_Subs', axis=1).to_excel('Shark Tank India S1 Data with Subtitles.xlsx', index=False)